In [2]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [35]:
df = pd.read_excel('data.xlsx')
df['W_PCT'] = round((df['W'] + 0.5 * df['T']) / (df['W'] + df['L'] + df['T']) * 100, 1)
df.head()

,YEAR,RANK,TEAM,MANAGER,W,L,T,CLINCHED_PLAYOFFS,PTS_FOR,PTS_AGNST,WAIVER,MOVES,STREAK,W_PCT
0,2021,1,Your Coach,Marsh,7,7,0,Y,1841.36,1790.30,7,31,W-2,50.0
1,2021,2,#Trash_Commish,Masi,9,5,0,Y,1978.36,1656.44,10,24,W-1,64.3
2,2021,3,Golden Rams Fam,Shoey,10,4,0,Y,1824.06,1799.68,8,15,W-2,71.4
3,2021,4,Shmo Stars,Zach,6,8,0,Y,1767.56,1727.72,9,26,W-4,42.9
4,2021,5,AngelLikeMoniBoloni,Moni,9,5,0,Y,1841.04,1760.78,2,17,W-1,64.3


# Average rank

In [36]:
df_average_rank = df.copy()
df_average_rank = df_average_rank.groupby(['MANAGER'])['RANK'].mean().to_frame().reset_index()
with open('../src/data/average_rank.json', 'w') as fout:
    json.dump(df_average_rank.to_dict(orient='list'), fout, indent=2)

In [37]:
fig = px.bar(df_average_rank, x='MANAGER', y='RANK')
fig.update_layout({
    'showlegend': False,
    'xaxis': {
        'title': 'Manager'
    },
    'yaxis': {
        'title': 'Average Rank'
    }
})

# Most top 3 finishes

In [38]:
df_placed = df.copy()
df_placed = df_placed[df_placed['RANK'] <= 3]
df_placed = df_placed.groupby(['MANAGER', 'RANK']).size().unstack(fill_value=0).reset_index()
df_placed = df_placed.rename(columns={1: '1st', 2: '2nd', 3: '3rd'})
with open('../src/data/top_three_finishes.json', 'w') as fout:
    json.dump(df_placed.to_dict(orient='list'), fout, indent=2)

In [39]:
fig = px.bar(df_placed, x='MANAGER', y=['1st', '2nd', '3rd'])
fig.update_layout({
    'yaxis': {
        'title': 'Rank'
    }, 
    'legend': {
        'title': ''
    }
})

fig.show()

# Average points for and against

In [42]:
def get_average(data, stat):
    df = data.copy()
    df = df.groupby('MANAGER')[stat].mean().to_frame().reset_index()
    return df

frames = []
for i in ['PTS_FOR', 'PTS_AGNST', 'W', 'L', 'W_PCT', 'WAIVER', 'MOVES']:
    df_stat = get_average(df, i)
    frames.append(df_stat)

df_averages = pd.concat(frames, axis=1)
df_averages = df_averages.loc[:, ~df_averages.columns.duplicated()]
df_averages

,MANAGER,PTS_FOR,PTS_AGNST,W,L,W_PCT,WAIVER,MOVES
0,?,1450.740000,1646.740000,3.000,10.000,23.100000,1.000000,19.000000
1,AK,1632.652500,1629.065000,6.875,6.250,52.462500,7.500000,22.125000
2,Burg,1626.020000,1629.402500,6.625,6.500,50.687500,5.500000,12.625000
3,Ciggy,1639.392500,1658.237500,5.875,7.250,44.800000,5.000000,20.500000
4,Dane,429.840000,1530.180000,1.000,12.000,7.700000,1.000000,3.000000
5,Het,1548.495000,1600.775000,4.250,8.750,32.700000,4.500000,23.750000
6,Jake,1604.525000,1692.125000,5.750,7.500,43.275000,2.750000,9.250000
7,Kyle,1526.146667,1589.473333,7.000,6.000,53.833333,8.666667,16.333333
8,Marsh,1726.972500,1634.400000,7.000,6.125,53.337500,6.125000,21.375000
9,Masi,1720.477500,1562.485000,8.000,5.125,60.900000,7.375000,25.750000


In [43]:
def get_max(data, stat):
    df = data.copy()
    df = df.groupby('MANAGER')[stat].max().to_frame().reset_index()
    return df

frames = []
for i in ['PTS_FOR', 'PTS_AGNST', 'W', 'L', 'W_PCT', 'WAIVER', 'MOVES']:
    df_stat = get_max(df, i)
    frames.append(df_stat)

df_maxes = pd.concat(frames, axis=1)
df_maxes = df_maxes.loc[:, ~df_maxes.columns.duplicated()]
df_maxes

,MANAGER,PTS_FOR,PTS_AGNST,W,L,W_PCT,WAIVER,MOVES
0,?,1450.74,1646.74,3,10,23.1,1,19
1,AK,1757.68,1811.98,8,8,61.5,10,27
2,Burg,1750.12,1837.38,9,10,69.2,12,17
3,Ciggy,1870.04,1786.18,10,9,76.9,8,29
4,Dane,429.84,1530.18,1,12,7.7,1,3
5,Het,1602.22,1672.56,7,10,53.8,5,29
6,Jake,1720.46,1810.64,9,12,69.2,5,13
7,Kyle,1563.28,1623.42,8,7,61.5,11,25
8,Marsh,1962.32,1790.30,9,11,69.2,9,31
9,Masi,1978.36,1658.60,10,8,76.9,10,34


In [49]:
df_clinched_playoffs = df.copy()
df_clinched_playoffs = df_clinched_playoffs[df_clinched_playoffs['CLINCHED_PLAYOFFS'] == 'Y']
df_clinched_playoffs = df_clinched_playoffs.groupby(['MANAGER']).size().reset_index()
df_clinched_playoffs = df_clinched_playoffs.rename(columns={0: 'Count'})
df_clinched_playoffs

,MANAGER,Count
0,AK,5
1,Burg,5
2,Ciggy,1
3,Jake,3
4,Kyle,3
5,Marsh,6
6,Masi,7
7,Moni,5
8,Shoey,2
9,Stew,5
